In [6]:
# -*- coding:utf-8 -*-

import tkinter
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg,NavigationToolbar2TkAgg

#生成0x400模式下的能量list
def elist_0x400(addr):
    with open(addr, 'rb') as f:
        blk = f.read(2)
        f.seek(2, 1)
        run_type = f.read(2)
        f.seek(16, 0)
        event_length0 = int.from_bytes(f.read(2), 'little') * 64
        event_length1 = int.from_bytes(f.read(2), 'little') * 64
        event_length2 = int.from_bytes(f.read(2), 'little') * 64
        event_length3 = int.from_bytes(f.read(2), 'little') * 64

        if run_type == b'\x00\x04':
            f.seek(64, 0)
            f.seek(16, 1)
            energy = f.read(2)
            channo = f.read(2)
            e0, e1, e2, e3 = [], [], [], []
        else:
            return None
        while channo:
            energy_d = int.from_bytes(energy, 'little')
            channo_d = int.from_bytes(channo, 'little')
            if channo_d == 0:
                e0.append(energy_d)
                f.seek(event_length0-4, 1)
                energy = f.read(2)
                channo = f.read(2)
            elif channo_d == 1:
                e1.append(energy_d)
                f.seek(event_length1-4, 1)
                energy = f.read(2)
                channo = f.read(2)
            elif channo_d == 2:
                e2.append(energy_d)
                f.seek(event_length2-4, 1)
                energy = f.read(2)
                channo = f.read(2)
            elif channo_d == 3:
                e3.append(energy_d)
                f.seek(event_length3-4, 1)
                energy = f.read(2)
                channo = f.read(2)
        return e0, e1, e2, e3
     
#生成能谱图2
def energy_spec2(addr):
    e00, e01, e02, e03 = elist_0x400(addr)
    e0 = list(map(lambda x:x/64, e00))
    e1 = list(map(lambda x:x/64, e01))
    e2 = list(map(lambda x:x/64, e02))
    e3 = list(map(lambda x:x/64, e03))
    
    fig = plt.figure(figsize=(16, 12))
 
    num_bins = 1024
    n0, bins0, patches0 = plt.hist(e0, num_bins, range=(0, num_bins), facecolor='green', alpha=50, label='ch0')
    n1, bins1, patches1 = plt.hist(e1, num_bins, range=(0, num_bins), facecolor='blue', alpha=50, label='ch1')
    n2, bins2, patches2 = plt.hist(e2, num_bins, range=(0, num_bins), facecolor='red', alpha=50, label='ch2')
    n3, bins3, patches3 = plt.hist(e3, num_bins, range=(0, num_bins), facecolor='yellow', alpha=50, label='ch3')
    plt.legend(prop={'size': 15})

    return fig
    

#生成能谱图
def energy_spec(addr):
    #读取文件，并把Energy取出做成一个list
    file = pd.read_csv(addr, skiprows=[0,1])
    energy = list(file['Energy'])
    e = list(map(lambda x:x/128, energy))

    fig = plt.figure(figsize=(16, 12))
 
    num_bins = 1024
    n, bins, patches = plt.hist(e, num_bins, range=(0, num_bins), facecolor='green', alpha=30)

    return fig

#生成PSA谱图
def PSA_spec(addr, scal):
    #读取文件，并把Energy取出做成一个list
    file = pd.read_csv(addr, skiprows=[0,1])
    psav = list(file['PSAvalue'])
    v = list(map(lambda x:x/scal, psav))

    fig = plt.figure(figsize=(16, 12))
 
    num_bins = 256
    n, bins, patches = plt.hist(v, num_bins, range=(0, num_bins), facecolor='green', alpha=30)

    return fig

#产生谱图的新窗口
def plot(b, scal):
    global addrs
    plot = tkinter.Tk()
    if b == 'b2':
        figs = energy_spec(addrs)
    elif b == 'b3':
        figs = PSA_spec(addrs,scal)
    elif b == 'b4':
        figs = energy_spec2(addrs)
    canvas = FigureCanvasTkAgg(figs, master=plot)
    canvas.show()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)
    plot.mainloop()
    
#产生PSA谱的刻度值填写窗口
def scale_value(b):
    def start():
        s = int(en.get())
        plot(b, s)
    window = tkinter.Tk()
    
    lb = tkinter.Label(window, text='请输入合适的刻度值')
    lb.grid(row=0, column=0)
    
    en = tkinter.Entry(window)
    en.grid(row=1, column=0)
    
    b1 = tkinter.Button(window, text='start', command=start)
    b1.grid(row=2, column=0)
    
    window.mainloop()

if __name__ == "__main__":
    spec = tkinter.Tk()
    def xz():
        global lb1, addrs
        filename = tkinter.filedialog.askopenfilename()
        addrs = filename
        if filename != '':
            lb1.config(text = filename)
        else:
            lb1.config(text = "您没有选择任何文件")
    
    lb1 = tkinter.Label(spec, text="请选择一个文件")
    lb1.grid(row=0, column=0, stick=tkinter.W )
    
    b1 = tkinter.Button(spec,text='选择502模式文件',command=xz)
    b1.grid(row=1, column=0)
    
    b12 = tkinter.Button(spec,text='选择400模式文件',command=xz)
    b12.grid(row=2, column=0)

    b2 = tkinter.Button(spec,text='start energy_plot',command=lambda:plot(b='b2',scal=1))
    b2.grid(row=3, column=0)
    
    b3 = tkinter.Button(spec,text='start PSA_plot',command=lambda:scale_value(b='b3'))
    b3.grid(row=4, column=0)
    
    b4 = tkinter.Button(spec,text='start energy_plot2',command=lambda:plot(b='b4',scal=1))
    b4.grid(row=5, column=0)
    
    spec.mainloop()

